matplotlib.pyplot → plot images and masks

rasterio → read GeoTIFF images

numpy → handle arrays

random → select random samples

Path → handle file paths cleanly

In [ ]:
import matplotlib.pyplot as plt
import rasterio
import numpy as np
import random
from pathlib import Path


Organizes paths and sample size in one place.

Creates the output directory for saving visualizations if it doesn’t exist.

In [ ]:
# Base project directory (update as needed)
BASE_PATH = Path("/content/cafo_project")

# Directories
IMG_DIR = BASE_PATH / "patches/images"
MASK_DIR = BASE_PATH / "patches/masks"
OUTPUT_VIS_DIR = BASE_PATH / "patch_visualizations"
OUTPUT_VIS_DIR.mkdir(parents=True, exist_ok=True)

# How many random samples to visualize
N_SAMPLES = 5


Collects all image patch files.

Randomly selects a subset to visualize.

In [ ]:
# Get all image patch files
image_files = sorted(IMG_DIR.glob("*.tif"))

# Randomly select N_SAMPLES patches
sample_files = random.sample(image_files, min(N_SAMPLES, len(image_files)))


Shows the selected image–mask pairs in a grid.

Helps visually inspect if patches and masks are aligned.

In [ ]:
fig, axes = plt.subplots(len(sample_files), 2, figsize=(8, 4 * len(sample_files)))

for i, img_path in enumerate(sample_files):
    mask_path = MASK_DIR / img_path.name.replace(".tif", "_mask.tif")

    with rasterio.open(img_path) as src:
        img = src.read([1, 2, 3]).transpose(1, 2, 0) / 255.0  # Convert to HWC, normalize

    with rasterio.open(mask_path) as src:
        mask = src.read(1)

    axes[i, 0].imshow(img)
    axes[i, 0].set_title(f"Image: {img_path.name}")
    axes[i, 0].axis("off")

    axes[i, 1].imshow(mask, cmap="gray")
    axes[i, 1].set_title(f"Mask: {mask_path.name}")
    axes[i, 1].axis("off")


Saves each random image–mask pair to a PNG file.

Can be used for documentation or quick QA checks.

In [ ]:
for img_path in sample_files:
    out_path = OUTPUT_VIS_DIR / f"{img_path.stem}_pair.png"
    fig, axes = plt.subplots(1, 2, figsize=(8, 4))

    mask_path = MASK_DIR / img_path.name.replace(".tif", "_mask.tif")

    with rasterio.open(img_path) as src:
        img = src.read([1, 2, 3]).transpose(1, 2, 0) / 255.0
    with rasterio.open(mask_path) as src:
        mask = src.read(1)

    axes[0].imshow(img)
    axes[0].set_title("Image")
    axes[0].axis("off")

    axes[1].imshow(mask, cmap="gray")
    axes[1].set_title("Mask")
    axes[1].axis("off")

    plt.tight_layout()
    plt.savefig(out_path, dpi=200)
    plt.close()

print(f"✅ Saved {len(sample_files)} patch pair visualizations to {OUTPUT_VIS_DIR}")
